In [12]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats
import re
import plotly.express as px
import config
from fredapi import Fred
import os
import shutil
from fpdf import FPDF

# User will need a FRED API key, can sign up for one here https://fred.stlouisfed.org/docs/api/api_key.html
# replace with your own API key: fred = Fred(api_key='YOUR KEY HERE')
fred = Fred(api_key=config.api_key)

In [13]:
from datetime import datetime, timedelta, date

class plot_obj:
  def __init__(self, df, title, units, notes, years_back, height = 400, width = 500):
    
    end_date_plot = df.index.max()
    start_date_plot = end_date_plot - timedelta(days = (365*years_back))
    self.df = df.loc[start_date_plot:end_date_plot]
    
    self.title = title
    self.units = units
    self.notes = notes
    self.height = height
    self.width = width
    
  
  def plot_clean(self):
    colnames = self.df.columns
    fig = px.line(self.df, x=self.df.index, y=colnames, title=self.title, template="plotly_white", width = self.width, height = self.height )
    fig.update_layout(
        
        font_family="Georgia",
        font_color="black",
        yaxis_title = self.units,
        xaxis_title = "Date"
    )
    fig.update_traces(connectgaps=True)
    fig.update_traces(line_color='#437070', line_width=2)
    fig.update_layout(showlegend=False)
    return fig



In [18]:
#Define a function to retrieve series, store as objects, and plot data 
def fredget(varlist,start_date, end_date, plots=True,freq="",name_map = dict()):
    df_collect = pd.DataFrame()
    plot_obs = list()
    idx = 0
    
    PLOT_DIR = 'plots/fredget'
    if plots == True:
        try:
            shutil.rmtree(PLOT_DIR)
            os.mkdir(PLOT_DIR)
        except FileNotFoundError:
            os.mkdir(PLOT_DIR)
        
    for var in varlist:
        #series_add = fred.get_series(var, observation_start=s_d, observation_end=e_d,frequency ='m')
        df_add = fred.get_series(var, observation_start = start_date, observation_end=end_date, frequency =freq).to_frame(name = var)
        info_add = fred.get_series_info(var).to_frame(name = var)
        
        if var in name_map:
            info_add.loc['title', var] = name_map[var]
            
        plot_obs.append(plot_obj(df_add, info_add.loc['title', var], info_add.loc['units', var] ,"notes",5))
        
        print(var)
        if plots == True:
            #df_add.plot(y=var, use_index=True, kind='line', title = info_add.loc['title', var], ylabel = info_add.loc['units', var] )
            fig = plot_obs[idx].plot_clean()
            fig.show()
            fig.write_image(f"{PLOT_DIR}/fig{idx}.png",scale=3, width=500, height=400)

            
        idx = idx + 1 
        if df_collect.empty == True:
            df_collect = df_add
            info_store = info_add
        else:
            df_collect = pd.merge(df_collect, df_add, left_index=True, right_index=True, how='outer')
            info_store = pd.merge(info_store, info_add, left_index=True, right_index=True, how='left')
            
    df_collect.sort_index()
    return df_collect,info_store



In [19]:

#Set Options Here:
start_d ='1982-01-01'
end_d='2023-02-03'
freq = "m"
FRED_list = ['T10Y2Y','WALCL','EFFR','SP500','MORTGAGE30US','BAMLH0A0HYM2']
Name_remap = {'T10Y2Y': '10-Year Minus 2-Year Treasury', 'WALCL': 'Reserve Balances: Total Assets'}

#Optional inputs: plots = True
df,dfinfo = fredget(FRED_list,start_d,end_d,name_map = Name_remap)

T10Y2Y


WALCL


EFFR


SP500


MORTGAGE30US


BAMLH0A0HYM2


In [20]:
#CODE BORROWED FROM THIS MEDIUM ARTICLE:
#LINK: https://towardsdatascience.com/how-to-create-pdf-reports-with-python-the-essential-guide-c08dd3ebf2ee

class PDF(FPDF):
    def __init__(self):
        super().__init__()
        self.WIDTH = 210
        self.HEIGHT = 297
        
    def header(self):
        # Custom logo and positioning
        # Create an `assets` folder and put any wide and short image inside
        # Name the image `logo.png`
        self.image('assets/logo.png', 10, 8, 50)
        self.set_font('Times', 'B', 12)
        self.cell(self.WIDTH - 80)
        self.cell(60, 16, 'Financial Markets Overview', 0, 0, 'R')
        self.ln(20)
        
    def footer(self):
        # Page numbers in the footer
        self.set_y(-15)
        self.set_font('Arial', 'I', 12)
        self.set_text_color(128)
        self.cell(0, 10, 'Page ' + str(self.page_no()), 0, 0, 'C')

    def page_body(self, images):
        # Determine how many plots there are per page and set positions
        # and margins accordingly
        if len(images) == 6:
            self.image(images[0], 15, 25, self.WIDTH - 30)
            self.image(images[1], 15, self.WIDTH / 2 + 5, self.WIDTH - 30)
            self.image(images[2], 15, self.WIDTH / 2 + 90, self.WIDTH - 30)
            self.image(images[3], 100, 25, self.WIDTH - 30)
            self.image(images[4], 100, self.WIDTH / 2 + 5, self.WIDTH - 30)
            self.image(images[5], 100, self.WIDTH / 2 + 90, self.WIDTH - 30)
            
        if len(images) == 3:
            self.image(images[0], 15, 25, self.WIDTH - 30)
            self.image(images[1], 15, self.WIDTH / 2 + 5, self.WIDTH - 30)
            self.image(images[2], 15, self.WIDTH / 2 + 90, self.WIDTH - 30)
        elif len(images) == 2:
            self.image(images[0], 15, 25, self.WIDTH - 30)
            self.image(images[1], 15, self.WIDTH / 2 + 5, self.WIDTH - 30)
        else:
            self.image(images[0], 15, 25, self.WIDTH - 30)
            
    def print_page(self, images):
        # Generates the report
        self.add_page()
        self.page_body(images)

In [21]:
pdf = PDF()

plots_per_page = [['plots/fredget/fig0.png','plots/fredget/fig1.png','plots/fredget/fig2.png','plots/fredget/fig3.png','plots/fredget/fig4.png','plots/fredget/fig5.png']]
for elem in plots_per_page:
    pdf.print_page(elem)
    
pdf.output('MacroReport.pdf', 'F')

''